# **Importamos las dependencias**

Estoy importando todas las dependencias de mis cuadernos, despues las depuro

In [105]:
# --- Importaciones para Manejo de Archivos y Directorios ---
import os
import sys
import subprocess
import re

# --- Importamos Vertex AI ---
import vertexai


# --- Importaciones para Visualización y Formato de Texto ---

# Se utiliza para mostrar contenido enriquecido, como texto con formato Markdown,
# directamente en entornos como Jupyter Notebooks o IPython.
from IPython.display import Markdown, display

# Importa la clase `Markdown` de la biblioteca `rich`, que sirve para renderizar
# Markdown con formato avanzado en la terminal. Se le da un alias `RichMarkdown`
# para evitar conflictos de nombre con la importación anterior.
from rich.markdown import Markdown as RichMarkdown


# --- Importaciones para el Modelo Generativo de Vertex AI ---

# Importa las clases necesarias del SDK de Vertex AI para interactuar con los modelos generativos.
# - GenerationConfig: Para configurar los parámetros de la respuesta (ej. temperatura, top_p).
# - GenerativeModel: La clase principal para cargar y usar un modelo generativo como Gemini.
# - Image: Para manejar y enviar imágenes como parte de la entrada al modelo (enfoque multimodal).
from vertexai.generative_models import GenerationConfig, GenerativeModel, Image

In [90]:
# --- Para el proceso de datos y visualización ---
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import time


# --- Para manejar las variables de entorno ---
import os
from dotenv import load_dotenv


# --- Desactiva las advertencias de asignaciones encadenadas en pandas para evitar mensajes de warning al modificar DataFrames.
pd.options.mode.chained_assignment = None  # default='warn'


# --- Dependencias de Vertex AI ---
import vertexai                                              # Importa el módulo principal de Vertex AI.
from vertexai import init                                    # Inicializa Vertex AI con las credenciales y configuraciones necesarias.
from vertexai.vision_models import Image as VMImage          # Importa la clase Image de Vertex AI para manejar imágenes.
from vertexai.vision_models import MultiModalEmbeddingModel  # Importa el modelo de embeddings multimodales de Vertex AI para procesar imágenes y videos.
from vertexai.vision_models import Video                     # Clase para manejar archivos de video en Vertex AI.
from vertexai.vision_models import VideoSegmentConfig        # Configuración para segmentar videos
from vertexai.generative_models import GenerativeModel       # Importa la clase para modelos generativos, como Gemini.
from vertexai.generative_models import Part                  # Importa la clase Part para manejar partes de un mensaje, como texto o imágenes.


# --- Para conectarse y consultar un endpoint de búsqueda vectorial (Vector Search) en Vertex AI. 
from google.cloud.aiplatform.matching_engine import MatchingEngineIndexEndpoint 

# --- Para acceder a los buckets de Google Cloud Storage y manejar archivos.
from google.cloud import storage


# --- Dependencias para poder visualizar ---
from IPython.display import Video as MVideo                  # Permite mostrar videos directamente en celdas de Jupyter Notebook.
from IPython.display import HTML                             # Permite mostrar contenido HTML en celdas de Jupyter Notebook.
from IPython.display import Image as ImageByte               # Permite mostrar imágenes en el notebook (renombrado como ImageByte para evitar conflictos de nombres).
from IPython.display import display                          # Función general para mostrar objetos en el notebook.
from sklearn.metrics.pairwise import cosine_similarity       # Función para calcular la similitud coseno entre vectores, útil para comparar embeddings.
# --- Para el proceso de datos y visualización ---
import numpy as np
import pandas as pd
import json 
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.mode.chained_assignment = None  # default='warn'
from google.cloud import storage



# --- Dependencias de Vertex AI ---
import vertexai                                              # Importa el módulo principal de Vertex AI.
from vertexai import init                                    # Inicializa Vertex AI con las credenciales y configuraciones necesarias.
from vertexai.vision_models import MultiModalEmbeddingModel  # Importa el modelo de embeddings multimodales de Vertex AI para procesar imágenes y videos.
from vertexai.vision_models import Video                     # Clase para manejar archivos de video en Vertex AI.
from vertexai.vision_models import VideoSegmentConfig        # Configuración para segmentar videos al gener
from google.cloud.aiplatform.matching_engine import MatchingEngineIndexEndpoint 
from vertexai.vision_models import Image

# --- Dependencias para poder visualizar ---
from IPython.display import Video as MVideo                  # Permite mostrar videos directamente en celdas de Jupyter Notebook.
from IPython.display import HTML                             # Permite mostrar contenido HTML en celdas de Jupyter Notebook.
from IPython.display import Image as ImageByte               # Permite mostrar imágenes en el notebook (renombrado como ImageByte para evitar conflictos de nombres).
from IPython.display import display                          # Función general para mostrar objetos en el notebook.
from sklearn.metrics.pairwise import cosine_similarity   

# **Configuración de credenciales**

In [106]:
# --- Carga las Variables de Entorno ---
load_dotenv()

PROJECT_ID = os.getenv("PROJECT_ID")                        # ID del proyecto de Google Cloud
LOCATION = os.getenv("LOCATION")                            # Región donde se encuentran los recursos de Vertex AI
INDEX_ID = os.getenv("INDEX_ID")                            # ID del índice de búsqueda vectorial en Vertex AI
ENDPOINT_ID = os.getenv("ENDPOINT_ID")                      # ID del endpoint de búsqueda vectorial en Vertex AI
BUCKET_NAME = os.getenv("BUCKET_NAME")                      # Nombre del bucket de Google Cloud Storage donde se almacenan los videos
VIDEO_FOLDER_PATH = os.getenv("VIDEO_FOLDER_PATH")          # Ruta del folder dentro del bucket donde se encuentran los videos

# Verificamos que las variebles de entorno esten bien
print(f"Proyecto: {PROJECT_ID}, Ubicación: {LOCATION}, Índice: {INDEX_ID}, Endpoint: {ENDPOINT_ID}, Bucket: {BUCKET_NAME}, Ruta de Videos: {VIDEO_FOLDER_PATH}")

Proyecto: None, Ubicación: None, Índice: None, Endpoint: None, Bucket: None, Ruta de Videos: None


In [120]:
PROJECT_ID="constant-setup-463820-p6"
LOCATION='us-central1'
INDEX_ID = "projects/640283206292/locations/us-central1/indexEndpoints/1468492336894836736"
ENDPOINT_ID = "mexicocostarica_1750850443317"
VIDEO_FOLDER_PATH = 'Videos/Videos_Segmentados'


In [107]:
# Iniciamos Vertex AI con el proyecto y la ubicación especificados.
init(project = PROJECT_ID, location = LOCATION)

# **Configuración de Vertex AI**

In [108]:
# --- Inicialización del LLM ---
model = GenerativeModel('gemini-2.5-flash-lite-preview-06-17')
embedding_model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding@001")

# **Funciones**

In [94]:
!gsutil ls

gs://vboxiooof/


In [181]:
from google.cloud import aiplatform_v1
from vertexai.vision_models import MultiModalEmbeddingModel
import re

# Configuración con tus valores exactos
API_ENDPOINT = "1584477394.us-central1-640283206292.vdb.vertexai.goog"
INDEX_ENDPOINT = "projects/640283206292/locations/us-central1/indexEndpoints/1468492336894836736"
DEPLOYED_INDEX_ID = "mexicocostarica_1750850443317"
BUCKET_NAME = "vboxiooof"
VIDEO_FOLDER_PATH = "Videos/Videos_Segmentados"

# Inicialización del modelo de embeddings (hazlo solo una vez)
embedding_model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding@001")

def buscar_videos_similares(
    texto_pregunta: str,
    num_resultados: int = 1
) -> list[str]:
    """
    Implementación basada en la documentación oficial de Vertex AI
    
    Args:
        texto_pregunta: Texto para convertir en embedding y buscar similitudes
        num_resultados: Número de videos similares a retornar
        
    Returns:
        Lista de URIs de videos encontrados (ej. ["gs://vboxiooof/Videos/video_001.mkv"])
    """
    print("🔍 Paso 1/3: Generando embedding del texto...")
    try:
        # Generar embedding del texto
        embedding = embedding_model.get_embeddings(
            contextual_text=texto_pregunta
        ).text_embedding
    except Exception as e:
        print(f"❌ Error al generar embedding: {e}")
        return []

    print("🔗 Paso 2/3: Configurando cliente de Vector Search...")
    try:
        # Configurar cliente como indica la documentación
        client_options = {"api_endpoint": API_ENDPOINT}
        client = aiplatform_v1.MatchServiceClient(client_options=client_options)
    except Exception as e:
        print(f"❌ Error al configurar cliente: {e}")
        return []

    print(f"📡 Paso 3/3: Buscando {num_resultados} videos similares...")
    try:
        # Construir solicitud como en la documentación
        datapoint = aiplatform_v1.IndexDatapoint(
            datapoint_id="query_embedding",
            feature_vector=embedding
        )

        query = aiplatform_v1.FindNeighborsRequest.Query(
            datapoint=datapoint,
            neighbor_count=num_resultados
        )

        request = aiplatform_v1.FindNeighborsRequest(
            index_endpoint=INDEX_ENDPOINT,
            deployed_index_id=DEPLOYED_INDEX_ID,
            queries=[query],
            return_full_datapoint=True  # Necesario para obtener los IDs
        )

        # Ejecutar la solicitud
        response = client.find_neighbors(request)

        # Procesar resultados
        uris = []
        for neighbor in response.nearest_neighbors[0].neighbors:
            video_id = neighbor.datapoint.datapoint_id
            try:
                if '_' in video_id:
                    prefix, num = video_id.rsplit('_', 1)
                    uri = f"/{prefix}_{num.zfill(3)}"
                    uris.append(uri)
                    print(f"✅ Embedding encontrado: {uri}")
            except Exception as e:
                print(f"⚠️ Error procesando {video_id}: {e}")

        return uris

    except Exception as e:
        print(f"❌ Error en la búsqueda vectorial: {e}")
        return []

In [182]:
#version modificada para manejar correctamente los segmentos de video



def analizar_fragmentos_con_gemini(
    pregunta_original: str,
    uris_de_videos: list[str]
) -> str:
    """
    Versión final que maneja correctamente el formato de 3 dígitos en los segmentos
    
    Args:
        pregunta_original: Pregunta del usuario
        uris_de_videos: Lista de URIs encontradas por Vector Search
        
    Returns:
        Respuesta generada por Gemini basada en los videos
    """
    if not uris_de_videos:
        return "No se encontraron videos relevantes para analizar."

    print(f"\n📦 Procesando {len(uris_de_videos)} URIs de video...")
    
    # Procesamos las URIs para obtener los segmentos reales (3 dígitos)
    videos_reales = set()  # Usamos un set para evitar duplicados
    

    #Funcion para obtener el numero real del video
    for uri in uris_de_videos:
        try:
            # Extraemos el número de segmento del URI
            nombre_archivo = uri.split('/')[-1].replace('.mkv', '')
            base, num_str = nombre_archivo.rsplit('_', 1)
            num = int(num_str)
            
            # Calculamos el segmento real de video (cada video tiene 4 embeddings)
            segmento_real = num // 4
            
            # Formateamos a 3 dígitos (041 en lugar de 0041)
            uri_real = f"gs://{BUCKET_NAME}/{VIDEO_FOLDER_PATH}/{base}_{segmento_real:03d}.mkv"
            
            videos_reales.add(uri_real)
            print(f"🔍 Mapeado: {uri} -> {uri_real}")
            
        except Exception as e:
            print(f"⚠️ Error procesando {uri}: {e}")
            continue
    
    if not videos_reales:
        return "No se pudieron mapear los segmentos a videos reales."

    print(f"\n🎬 Videos reales a analizar ({len(videos_reales)}):")
    for video in videos_reales:
        print(f" - {video}")

    # Convertimos las URIs reales a partes para Gemini
    try:
        video_parts = [Part.from_uri(uri, mime_type="video/mkv") for uri in videos_reales]
        
        prompt = [
            "Eres un experto analista de video. Analiza ESTOS VIDEOS EXACTOS:",
            *video_parts,
            "\nPREGUNTA DEL USUARIO:",
            pregunta_original,
            "\nINSTRUCCIONES:",
            "1. Responde basado SOLO en los videos proporcionados",
            "2. Sé preciso y conciso",
            "3. Incluye referencias temporales cuando sea relevante",
            "4. Si no hay información, di 'No encontrado en los videos'"
        ]
        
        print("\n🧠 Enviando a Gemini para análisis...")
        response = model.generate_content(prompt)
        return response.text
        
    except Exception as e:
        return f"Error al analizar videos: {str(e)}"

# **Pruebas**

In [183]:

# --------------------------------------------------------------------------
# --- 4. EJECUCIÓN DEL FLUJO COMPLETO (Sin cambios) ---
# --------------------------------------------------------------------------
if __name__ == "__main__":
    pregunta_del_usuario = "Muestrame a la marca Caliente"
    print("--- INICIANDO PROCESO DE RAG MULTIMODAL ---")
    uris_relevantes = buscar_videos_similares(pregunta_del_usuario)
    respuesta_final = analizar_fragmentos_con_gemini(pregunta_del_usuario, uris_relevantes)
    print("\n--- RESPUESTA FINAL DE GEMINI ---")
    print(respuesta_final)

--- INICIANDO PROCESO DE RAG MULTIMODAL ---
🔍 Paso 1/3: Generando embedding del texto...
🔗 Paso 2/3: Configurando cliente de Vector Search...
📡 Paso 3/3: Buscando 1 videos similares...
✅ Embedding encontrado: /mexicosta_segment_1095

📦 Procesando 1 URIs de video...
🔍 Mapeado: /mexicosta_segment_1095 -> gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_273.mkv

🎬 Videos reales a analizar (1):
 - gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_273.mkv

🧠 Enviando a Gemini para análisis...

--- RESPUESTA FINAL DE GEMINI ---
Error al analizar videos: 404 POST https://us-central1-aiplatform.googleapis.com/v1/projects/constant-setup-463820-p6/locations/us-central1/publishers/google/models/gemini-2.5-flash-lite-preview-06-17:generateContent?%24alt=json%3Benum-encoding%3Dint: Publisher Model `projects/constant-setup-463820-p6/locations/us-central1/publishers/google/models/gemini-2.5-flash-lite-preview-06-17` was not found or your project does not have access to it. Please e

In [151]:
# Prueba mínima con un video conocido
test_uri = "gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_041.mkv"
test_response = analizar_fragmentos_con_gemini("Describe este video", [test_uri])


📦 Procesando 1 URIs de video...
🔍 Mapeado: gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_041.mkv -> gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_010.mkv

🎬 Videos reales a analizar (1):
 - gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_010.mkv

🧠 Enviando a Gemini para análisis...


In [ ]:
#PRUEBAS PARA DETERMINAR UMBRAL



from google.cloud import aiplatform_v1
from vertexai.vision_models import MultiModalEmbeddingModel
import re

# Configuración con tus valores exactos
API_ENDPOINT = "1584477394.us-central1-640283206292.vdb.vertexai.goog"
INDEX_ENDPOINT = "projects/640283206292/locations/us-central1/indexEndpoints/1468492336894836736"
DEPLOYED_INDEX_ID = "mexicocostarica_1750850443317"
BUCKET_NAME = "vboxiooof"
VIDEO_FOLDER_PATH = "Videos/Videos_Segmentados"

def buscar_videos_similares1(
    texto_pregunta: str,
    umbral_similaridad: float = 0.9,
    num_resultados_max: int = 100
) -> tuple[int, list[str]]:
    """
    Versión modificada que busca todos los resultados que superen un umbral de similaridad
    
    Args:
        texto_pregunta: Texto para convertir en embedding y buscar similitudes
        umbral_similaridad: Umbral de similaridad (0.0 a 1.0) - 0.7 = 70%
        num_resultados_max: Máximo número de resultados a considerar
        
    Returns:
        tuple: (cantidad_resultados, lista_de_uris) donde:
            - cantidad_resultados: Número total de videos que superaron el umbral
            - lista_de_uris: Lista de URIs de videos encontrados
    """
    print("🔍 Paso 1/3: Generando embedding del texto...")
    try:
        # Generar embedding del texto
        embedding = embedding_model.get_embeddings(
            contextual_text=texto_pregunta
        ).text_embedding
    except Exception as e:
        print(f"❌ Error al generar embedding: {e}")
        return (0, [])

    print("🔗 Paso 2/3: Configurando cliente de Vector Search...")
    try:
        client = aiplatform_v1.MatchServiceClient(
            client_options={"api_endpoint": API_ENDPOINT}
        )
    except Exception as e:
        print(f"❌ Error al configurar cliente: {e}")
        return (0, [])

    print(f"📡 Paso 3/3: Buscando videos con similaridad > {umbral_similaridad*100}%...")
    try:
        # Configuramos para obtener muchos resultados y luego filtrar
        datapoint = aiplatform_v1.IndexDatapoint(
            datapoint_id="query_embedding",
            feature_vector=embedding
        )

        query = aiplatform_v1.FindNeighborsRequest.Query(
            datapoint=datapoint,
            neighbor_count=num_resultados_max  # Obtenemos el máximo posible
        )

        request = aiplatform_v1.FindNeighborsRequest(
            index_endpoint=INDEX_ENDPOINT,
            deployed_index_id=DEPLOYED_INDEX_ID,
            queries=[query],
            return_full_datapoint=True
        )

        # Ejecutar la solicitud
        response = client.find_neighbors(request)

        # Procesar resultados aplicando el umbral
        resultados_validos = 0
        uris = []
        
        for neighbor in response.nearest_neighbors[0].neighbors:
            # Verificar similaridad (distance es 1 - similaridad)
            similaridad = 1 - neighbor.distance
            
            if similaridad >= umbral_similaridad:
                resultados_validos += 1
                video_id = neighbor.datapoint.datapoint_id
                try:
                    if '_' in video_id:
                        prefix, num = video_id.rsplit('_', 1)
                        uri = f"gs://{BUCKET_NAME}/{VIDEO_FOLDER_PATH}/{prefix}_{num.zfill(3)}.mkv"
                        uris.append(uri)
                        print(f"✅ Video encontrado (similaridad: {similaridad:.2f}): {uri}")
                except Exception as e:
                    print(f"⚠️ Error procesando {video_id}: {e}")

        print(f"\n🎯 Total de videos que superan el umbral: {resultados_validos}")
        return (resultados_validos, uris)

    except Exception as e:
        print(f"❌ Error en la búsqueda vectorial: {e}")
        return (0, [])


In [156]:
# Ejemplo 1: Obtener solo el conteo
cantidad, _ = buscar_videos_similares1("Muestrame la marca Caliente")
print(f"\nTotal de videos relevantes: {cantidad}")



🔍 Paso 1/3: Generando embedding del texto...
🔗 Paso 2/3: Configurando cliente de Vector Search...
📡 Paso 3/3: Buscando videos con similaridad > 90.0%...
✅ Video encontrado (similaridad: 0.90): gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_1004.mkv
✅ Video encontrado (similaridad: 0.90): gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_033.mkv
✅ Video encontrado (similaridad: 0.90): gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_164.mkv
✅ Video encontrado (similaridad: 0.90): gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_923.mkv
✅ Video encontrado (similaridad: 0.90): gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_026.mkv
✅ Video encontrado (similaridad: 0.90): gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_999.mkv
✅ Video encontrado (similaridad: 0.90): gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_490.mkv
✅ Video encontrado (similaridad: 0.90): gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segmen

In [168]:
def buscar_videos_similares2(
    texto_pregunta: str,
    umbral_similaridad: float = 0.91,
    num_resultados_max: int = 100
) -> tuple[int, list[str]]:
    """
    Versión final que aplica umbral de similaridad y divide segmentos entre 4
    
    Args:
        texto_pregunta: Texto para buscar videos similares
        umbral_similaridad: Mínima similaridad aceptada (0.7 = 70%)
        num_resultados_max: Máximo de resultados a considerar para filtrar
        
    Returns:
        tuple: (total_resultados, lista_uris)
            - total_resultados: Cantidad de videos que superaron el umbral
            - lista_uris: URIs de videos reales (segmentos divididos entre 4)
    """
    print("🔍 Paso 1/3: Generando embedding del texto...")
    try:
        embedding = embedding_model.get_embeddings(
            contextual_text=texto_pregunta
        ).text_embedding
    except Exception as e:
        print(f"❌ Error en embedding: {e}")
        return (0, [])

    print("🔗 Paso 2/3: Configurando cliente...")
    try:
        client = aiplatform_v1.MatchServiceClient(
            client_options={"api_endpoint": API_ENDPOINT}
        )
    except Exception as e:
        print(f"❌ Error en cliente: {e}")
        return (0, [])

    print(f"📡 Paso 3/3: Buscando videos (similaridad > {umbral_similaridad*100}%)...")
    try:
        # Configuración de búsqueda
        request = aiplatform_v1.FindNeighborsRequest(
            index_endpoint=INDEX_ENDPOINT,
            deployed_index_id=DEPLOYED_INDEX_ID,
            queries=[aiplatform_v1.FindNeighborsRequest.Query(
                datapoint=aiplatform_v1.IndexDatapoint(feature_vector=embedding),
                neighbor_count=num_resultados_max
            )],
            return_full_datapoint=True
        )

        response = client.find_neighbors(request)
        resultados_validos = 0
        uris_reales = set()  # Usamos set para evitar duplicados

        for neighbor in response.nearest_neighbors[0].neighbors:
            similaridad = 1 - neighbor.distance
            
            if similaridad >= umbral_similaridad:
                resultados_validos += 1
                video_id = neighbor.datapoint.datapoint_id
                
                try:
                    if '_' in video_id:
                        # Dividimos el número de segmento entre 4
                        prefix, num_str = video_id.rsplit('_', 1)
                        num_segmento = int(num_str)
                        num_video_real = num_segmento // 4
                        
                        # Formateamos a 3 dígitos (041)
                        uri_real = f"gs://{BUCKET_NAME}/{VIDEO_FOLDER_PATH}/{prefix}_{num_video_real:03d}.mkv"
                        uris_reales.add(uri_real)
                        
                        print(f"✅ Match (sim: {similaridad:.2f}): {video_id} → {uri_real}")
                
                except Exception as e:
                    print(f"⚠️ Error procesando {video_id}: {e}")

        print(f"\n🎯 Total sobre umbral: {resultados_validos}")
        print(f"🎬 Videos únicos encontrados: {len(uris_reales)}")
        
        return (resultados_validos, list(uris_reales))

    except Exception as e:
        print(f"❌ Error en búsqueda: {e}")
        return (0, [])

In [169]:
# Ejemplo 1: Obtener solo el conteo
cantidad, _ = buscar_videos_similares2("Muestrame la marca Caliente")
print(f"\nTotal de videos relevantes: {cantidad}")

🔍 Paso 1/3: Generando embedding del texto...
🔗 Paso 2/3: Configurando cliente...
📡 Paso 3/3: Buscando videos (similaridad > 91.0%)...
✅ Match (sim: 0.91): mexicosta_segment_988 → gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_247.mkv
✅ Match (sim: 0.91): mexicosta_segment_15 → gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_003.mkv
✅ Match (sim: 0.91): mexicosta_segment_201 → gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_050.mkv
✅ Match (sim: 0.91): mexicosta_segment_973 → gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_243.mkv
✅ Match (sim: 0.91): mexicosta_segment_978 → gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_244.mkv
✅ Match (sim: 0.91): mexicosta_segment_46 → gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_011.mkv
✅ Match (sim: 0.91): mexicosta_segment_1061 → gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_265.mkv
✅ Match (sim: 0.91): mexicosta_segment_20 → gs://vboxiooof/Videos/Videos_Segmentado